In [ ]:
import os
import sys
import subprocess
import signac
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statistics import linear_regression
from scipy.stats import linregress
from pymser import pymser

In [ ]:
def is_slope_zero(n, simulations_per_block, threshold=0.03):
    # Step 1: Divide the Data
    n_length = len(n)
    # print(f"Total number of data points: {n_length}")

    # Calculate the number of blocks
    m = n_length // simulations_per_block
    # print(f"Number of blocks: {m}")

    # Initialize averages list
    averages = []

    for i in range(m):
        start_index = i * simulations_per_block
        # Handle the last block to include all remaining elements
        end_index = n_length if i == m - 1 else (i + 1) * simulations_per_block
        # print(f"Block {i + 1}: Start index = {start_index}, End index = {end_index}")

        # Calculate the average of the current block
        section_average = np.mean(n[start_index:end_index])
        averages.append(section_average)

    # Step 2: Scale the Averages between 0 and 1
    scaler = StandardScaler()  # You can change to MinMaxScaler() if desired
    scaled_averages = scaler.fit_transform(np.array(averages).reshape(-1, 1)).flatten()

    # Step 3: Divide into 5 groups
    group_size = m // 5
    slopes = []
    intercepts = []
    
    #Make the first 3 1/5 and the last 2 2/5
    for j in range(5):
        group_start = j * group_size
        group_end = (j + 1) * group_size if j < 3 else m  # Last group gets any remaining blocks
        # print(f"Group {j + 1}: Start = {group_start}, End = {group_end}")
        if group_end - group_start < 2:  # Need at least two points to calculate slope
            print(f"Group {j + 1} has insufficient data points.")
            continue
        
        x_group = np.arange(group_start, group_end)
        y_group = scaled_averages[group_start:group_end]
        slope, intercept, r_value, p_value, std_err = linregress(x_group, y_group)
        slopes.append(slope)
        intercepts.append(intercept)
        # print(f"Group {j + 1}: Slope = {slope}, Intercept = {intercept}")

    # print(f"Slopes: {slopes}")   
    # Step 3: Fit a Line with intercept fixed at 0
    x = np.arange(m)
    slope, intercept, r_value, p_value, std_err = linregress(x, scaled_averages)
    
    # Step 4: Check the Slope
    # print(slope, slopes[-1], threshold)
    equilibrated = (abs(slope) < threshold and abs(slopes[-1]) < threshold) or abs(slopes[-2]) < threshold*(2/3)

    if equilibrated:
        # Plotting
        plt.figure(figsize=(10, 6))

        # Original Data
        plt.subplot(1, 2, 1)
        plt.plot(scaler.transform(n.reshape(-1, 1)), label='Original Data', color='blue')
        
        # Draw vertical lines for each block average
        for i in range(m):
            # Calculate the x position scaled based on total number of data
            x_position = (i * simulations_per_block + (simulations_per_block / 2))  # Middle of the block
            plt.axvline(x=x_position, color='orange', linestyle='--', lw=0.7,
                        label=f'Block {i + 1} Average' if i == 0 else "")
        
        for j in range(1, 4):  # Draw lines after each of the first two groups
            group_boundary_x = j * group_size * simulations_per_block - (simulations_per_block / 2)
            plt.axvline(x=group_boundary_x, color='black', linestyle='-', lw=2,
                        label=f'Group {j} Boundary' if j == 1 else "")
        plt.title('Original Data')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.legend()

        # Averages and Line of Best Fit
        plt.subplot(1, 2, 2)
        plt.scatter(x, scaled_averages, label='Block Averages', color='orange')
        
        # Plot the line of best fit with intercept fixed at 0
        plt.plot(x, slope * x + intercept, label='Line of Best Fit (Intercept = 0)', color='red')
        plt.title('Block Averages and Line of Best Fit')
        plt.xlabel('Block')
        plt.ylabel('Scaled Average')
        plt.axhline(y=0, color='k', linestyle='--', lw=0.7)  # y=0 line for reference
        plt.legend()

        plt.tight_layout()
        plt.show()
    
    return slope, slopes, equilibrated

In [ ]:
# Find all jobs with the specified statepoint value
mol_name = "R170"
project = signac.get_project("opt_ff_ms")
jobs = project.find_jobs({"mol_name": mol_name})# "T": float(sys.argv[3]), "atom_type": int(sys.argv[2])})

# Iterate over the matching jobs
for job in jobs:
    # Construct the command using the job ID and statepoint value
    print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
    try:
        df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))
        energy = df_box1[:, 2 - 1]

        
        slope, group_slopes, result = is_slope_zero(energy, 15)
        print(f"The slope of all {round(slope,4)} and last 1/5 {round(group_slopes[-1],4)} or last 2/5 {round(group_slopes[-2],4)} of data is approximately zero: {result}")
    except:
        pass

In [ ]:
# #From Barnabas. Only seems to work on data that is long enough to equilibrate

# def running_average(data_raw):
#   n = data_raw.shape[0]
#   m = data_raw.shape[1]
#   running_ave = np.zeros([n, m])
#   for i in range(m):
#     for j in range(n):
#       mov_average = data_raw[0:j+1,i].mean()
#       running_ave[j,i] = mov_average
#   return(running_ave)


# def relative_abs_error(a,b):
#   error = np.abs(a-b)
#   rae = np.abs(error/a)
#   return rae

# import numpy as np

# def relative_abs_error(a, b):
#     """Calculate the relative absolute error between two values."""
#     return abs(a - b) / max(abs(a), abs(b))

# def test_equilibration(running_average_data):
#   start_index_frac = 0.60
#   stop_index_frac = 0.99
#   test_index_array = np.arange(start_index_frac, stop_index_frac, 0.005)
#   test_index = test_index_array*len(running_average_data)
#   test_points_index = test_index.astype(int)
#   equilibration_point = 0
#   for i in range(len(test_points_index)):
#     check_index = test_points_index[i]
#     equilibration_point = check_index
#     check_range_len = ((1 - stop_index_frac) * len(running_average_data)) - 1
#     check_range = np.arange(1, check_range_len, 1)
#     check_range = check_range.astype(int)
#     error_found = False

#     for k in (check_range):
#         rae = relative_abs_error(running_average_data[check_index], running_average_data[(check_index+k)])
#         if rae > 5e-4:
#             error_found = True  # Set the flag if an error is found
#             break  # Exit the inner loop if an error is found
#         else:
#           continue
#     if error_found == False:
#         break  # Exit the outer loop if no error is found

#   equilibration_time_index = equilibration_point
#   rem = equilibration_time_index%5
#   to_add = 0
#   if rem != 0:
#     to_add = 5 - rem
#   final_equib_time = equilibration_time_index + to_add
#   final_equib_time = int(final_equib_time)
#   final_equib_time_stored = np.array([final_equib_time])*1000
#   return final_equib_time_stored, equilibration_point, test_points_index


# mol_name = "R14"
# project = signac.get_project("opt_ff_ms")
# jobs = project.find_jobs({"mol_name": mol_name})# "T": float(sys.argv[3]), "atom_type": int(sys.argv[2])})

# # Iterate over the matching jobs
# for job in jobs:
#     # Construct the command using the job ID and statepoint value
#     print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)

#     # try:
#     df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))
#     eq_col = df_box1[:, 2 - 1]
#     running_average_data = running_average(eq_col.reshape(-1,1))
#     # print(running_average_data)
#     results = test_equilibration(running_average_data)
#     final_equib_time_stored, equilibration_point, test_points_index = results
    
#     print("Equilibration point: ", equilibration_point)
#     fig, ax1 = plt.subplots(1, 1)

#     ax1.set_ylabel("Energy", color="black", fontsize=14, fontweight='bold')
#     ax1.set_xlabel("GEMC step", fontsize=14, fontweight='bold')

#     ax1.plot(range(len(eq_col)), 
#             eq_col, 
#             label = 'Raw data', 
#             color='blue')

#     ax1.plot(range(len(eq_col))[equilibration_point:], 
#             eq_col[equilibration_point:], 
#             label = 'Equilibrated data', 
#             color='red')

In [ ]:
def plot_res_pymser(job, eq_col, results, name, box_name):
    fig, [ax1, ax2] = plt.subplots(1, 2, gridspec_kw={'width_ratios': [2, 1]}, sharey=True)

    ax1.set_ylabel(name, color="black", fontsize=14, fontweight='bold')
    ax1.set_xlabel("GEMC step", fontsize=14, fontweight='bold')

    ax1.plot(range(len(eq_col)), 
            eq_col, 
            label = 'Raw data', 
            color='blue')

    ax1.plot(range(len(eq_col))[results['t0']:], 
            results['equilibrated'], 
            label = 'Equilibrated data', 
            color='red')

    ax1.plot([0, len(eq_col)], 
            [results['average'], results['average']], 
            color='green', zorder=4, 
            label='Equilibrated average')

    ax1.fill_between(range(len(eq_col)), 
                    results['average'] - results['uncertainty'], 
                    results['average'] + results['uncertainty'], 
                    color='lightgreen', alpha=0.3, zorder=4)

    ax1.set_yticks(np.arange(0, eq_col.max()*1.1, eq_col.max()/10))
    ax1.set_xlim(-len(eq_col)*0.02, len(eq_col)*1.02)
    ax1.tick_params(axis="y", labelcolor="black")

    ax1.grid(alpha=0.3)
    ax1.legend()

    ax2.hist(eq_col, 
            orientation=u'horizontal', 
            bins=30, 
            edgecolor='blue', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ax2.hist(results['equilibrated'], 
            orientation=u'horizontal', 
            bins=3, 
            edgecolor='red', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ymax = int(ax2.get_xlim()[-1])

    ax2.plot([0, ymax], 
            [results['average'], results['average']],
            color='green', zorder=4, label='Equilibrated average')

    ax2.fill_between(range(ymax), 
                    results['average'] - results['uncertainty'],
                    results['average'] + results['uncertainty'],
                    color='lightgreen', alpha=0.3, zorder=4)

    ax2.set_xlim(0, ymax)

    ax2.grid(alpha=0.5, zorder=1)

    fig.set_size_inches(9,5)
    fig.set_dpi(100)
    fig.tight_layout()
    save_name = 'MSER_eq_'+ box_name +'.png'
    # fig.savefig(job.fn(save_name), dpi=300, facecolor='white')
#     plt.close(fig)
    plt.show()

In [ ]:
import glob
mol_name = "R170"
verbose = True
plot_if_all_true = True
project = signac.get_project("opt_ff_ms")
jobs = project.find_jobs({"mol_name": mol_name, "T": 240, "atom_type": 8})
jobs = project.find_jobs({"mol_name": mol_name})
prop_cols = [5]
prop_names = ["Number of Moles"]

# prop_cols = [2,3, 4,5,6]
# prop_names = ["Energy (kJ/mol)", "Pressure (Pa)", "Volume (A^3)", "Moles", "Density (kg/m^3)"]

# prop_cols = [4,5,6]
# prop_names = ["Volume (A^3)", "Moles", "Density (kg/m^3)"]

# # Iterate over the matching jobs
# Iterate over the matching jobs
for job in jobs:
    # print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
    equil_matrix = []
    res_matrix = []
    try:
        # Load data for both boxes
        df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))
        df_box2 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))
        prod_cyc = len(df_box1) / 4
        # Process both boxes in one loop
        for box in [df_box1, df_box2]:
            for prop_index in prop_cols:
                eq_col = box[:, prop_index - 1]
                # print(len(eq_col))
                batch_size = max(1, int(len(eq_col) * 0.0005))

                # Try with ADF test enabled, fallback without it if it fails
                try:
                    results = pymser.equilibrate(eq_col, LLM=False, batch_size=batch_size, ADF_test=True, uncertainty='uSD', print_results=False)
                    adf_test_failed = results["critical_values"]["1%"] <= results["adf"]
                except:
                    results = pymser.equilibrate(eq_col, LLM=False, batch_size=batch_size, ADF_test=False, uncertainty='uSD', print_results=False)
                    results["adf"], results["critical_values"], adf_test_failed = None, None, False

                equilibrium = len(eq_col) - results['t0'] >= prod_cyc
                equil_matrix.append(equilibrium and not adf_test_failed)
                res_matrix.append(results)

        # Log results
        print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
        print(equil_matrix)
        log_text = '==============================================================================\n'
        
        for i, is_equilibrated in enumerate(equil_matrix):
            box = df_box1 if i < len(prop_cols) else df_box2
            box_name = "Liquid" if i < len(prop_cols) else "Vapor"
            col_vals = box[:, prop_cols[i % len(prop_cols)] - 1]
            #plot all

            # if not all(equil_matrix):
            plot_res_pymser(job, col_vals, res_matrix[i], prop_names[i % len(prop_cols)], box_name)

            # Display outcome
            prod_cycles = len(col_vals) - res_matrix[i]['t0']
            if is_equilibrated:
                #Plot successful equilibration
                # plot_res_pymser(col_vals, res_matrix[i], prop_names[i % len(prop_cols)])
                statement = f"       > Success! Found {prod_cycles} production cycles."
            else:
                #Plot failed equilibration
                statement = "       > Failure! "
                # plot_res_pymser(col_vals, res_matrix[i], prop_names[i % len(prop_cols)])
                if res_matrix[i]["adf"] is None:
                    # Note: ADF test failed to complete
                    statement += f"ADF test failed to complete! "
                elif res_matrix[i]['adf'] > res_matrix[i]['critical_values']['1%']:
                    adf, one_pct = res_matrix[i]['adf'], res_matrix[i]['critical_values']['1%']
                    statement += f"ADF value: {adf}, 99% confidence value: {one_pct}! "
                if len(col_vals) - res_matrix[i]['t0'] < prod_cyc/4:
                   statement += f"Only {prod_cycles} production cycles found."
                
            # if verbose:
            #     print(statement)

    except Exception as e:
        print(f"Error processing job {job.id}: {e}")

    #If all jobs pass, set nsteps_eq needed to the original amount
    if all(equil_matrix):
        pass
        # job.doc.nsteps_eq = job.sp.nsteps_eq
    #Otherwise, delete job production information, equil info will be retained and used since job.doc.nsteps_eq is not set 
    else:
        with job:
            print("Deleting production data")
            for file_path in glob.glob(os.path.join(job.fn(""), "prod.*")):
                # print("Deleting", file_path)
                # os.remove(file_path)
                pass



In [ ]:
print("gemc.eq" + f".rst.{1:03d}" + ".box2.prp")

In [ ]:
def plot_res_pymser(job, eq_col, results, name, box_name):
    fig, [ax1, ax2] = plt.subplots(1, 2, gridspec_kw={'width_ratios': [2, 1]}, sharey=True)

    ax1.set_ylabel(name, color="black", fontsize=14, fontweight='bold')
    ax1.set_xlabel("GEMC step", fontsize=14, fontweight='bold')

    ax1.plot(range(len(eq_col)), 
            eq_col, 
            label = 'Raw data', 
            color='blue')

    ax1.plot(range(len(eq_col))[results['t0']:], 
            results['equilibrated'], 
            label = 'Equilibrated data', 
            color='red')

    ax1.plot([0, len(eq_col)], 
            [results['average'], results['average']], 
            color='green', zorder=4, 
            label='Equilibrated average')

    ax1.fill_between(range(len(eq_col)), 
                    results['average'] - results['uncertainty'], 
                    results['average'] + results['uncertainty'], 
                    color='lightgreen', alpha=0.3, zorder=4)

    ax1.set_yticks(np.arange(0, eq_col.max()*1.1, eq_col.max()/10))
    ax1.set_xlim(-len(eq_col)*0.02, len(eq_col)*1.02)
    ax1.tick_params(axis="y", labelcolor="black")

    ax1.grid(alpha=0.3)
    ax1.legend()

    ax2.hist(eq_col, 
            orientation=u'horizontal', 
            bins=30, 
            edgecolor='blue', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ax2.hist(results['equilibrated'], 
            orientation=u'horizontal', 
            bins=3, 
            edgecolor='red', 
            lw=1.5, 
            facecolor='white', 
            zorder=3)

    ymax = int(ax2.get_xlim()[-1])

    ax2.plot([0, ymax], 
            [results['average'], results['average']],
            color='green', zorder=4, label='Equilibrated average')

    ax2.fill_between(range(ymax), 
                    results['average'] - results['uncertainty'],
                    results['average'] + results['uncertainty'],
                    color='lightgreen', alpha=0.3, zorder=4)

    ax2.set_xlim(0, ymax)

    ax2.grid(alpha=0.5, zorder=1)

    fig.set_size_inches(9,5)
    fig.set_dpi(100)
    fig.tight_layout()
    save_name = 'MSER_eq_'+ box_name +'.png'
    # fig.savefig(job.fn(save_name), dpi=300, facecolor='white')
    # plt.close(fig)

def check_equil_converge(job, eq_data_dict, prod_tol):
    equil_matrix = []
    res_matrix = []
    prop_cols = [5]
    prop_names = ["Number of Moles"]
    try:
        # Load data for both boxes
        for key in list(eq_data_dict.keys()):
            eq_col = eq_data_dict[key]["data"]
            batch_size = max(1, int(len(eq_col) * 0.0005))

            # Try with ADF test enabled, fallback without it if it fails
            try:
                results = pymser.equilibrate(eq_col, LLM=False, batch_size=batch_size, ADF_test=True, uncertainty='uSD', print_results=False)
                adf_test_failed = results["critical_values"]["1%"] <= results["adf"]
            except:
                results = pymser.equilibrate(eq_col, LLM=False, batch_size=batch_size, ADF_test=False, uncertainty='uSD', print_results=False)
                results["adf"], results["critical_values"], adf_test_failed = None, None, False

            equilibrium = len(eq_col) - results['t0'] >= prod_tol
            equil_matrix.append(equilibrium and not adf_test_failed)
            res_matrix.append(results)
        
        for i, is_equilibrated in enumerate(equil_matrix):
            key_name = list(eq_data_dict.keys())[i]
            box_name = key_name.rsplit("_", 1)[0]
            col_vals = eq_data_dict[key_name]["data"]
            #plot all

            # if not all(equil_matrix):
            plot_res_pymser(job, col_vals, res_matrix[i], prop_names[i % len(prop_cols)], box_name)

            # Display outcome
            prod_cycles = len(col_vals) - res_matrix[i]['t0']
            if is_equilibrated:
                #Plot successful equilibration
                statement = f"       > Success! Found {prod_cycles} production cycles."
            else:
                #Plot failed equilibration
                statement = f"       > {box_name} Box Failure! "
                if res_matrix[i]["adf"] is None:
                    # Note: ADF test failed to complete
                    statement += f"ADF test failed to complete! "
                elif res_matrix[i]['adf'] > res_matrix[i]['critical_values']['1%']:
                    adf, one_pct = res_matrix[i]['adf'], res_matrix[i]['critical_values']['1%']
                    statement += f"ADF value: {adf}, 99% confidence value: {one_pct}! "
                if len(col_vals) - res_matrix[i]['t0'] < prod_tol:
                   statement += f"Only {prod_cycles} production cycles found."
                
            print(statement)

    except Exception as e:
        #This will cause an error in the GEMC operation which lets us know that the job failed
        raise Exception(f"Error processing job {job.id}: {e}")

    return all(equil_matrix)

def run_gemc(job):
    "Run gemc"
    # Move into the job dir and start doing things
    use_crit = False
    try:
        #Inititalize counter and number of eq_steps
        count = 1
        total_eq_steps = job.sp.nsteps_eq
        prod_tol = int(job.sp.nsteps_eq/4)
        #Originally set the document eq_steps to 1 larger than the max number, it will be overwritten later
        total_steps = int(job.sp.nsteps_eq*4+1)
        with job:

            prop_cols = [5] #Use number of moles to decide equilibrium
            # Load initial eq data from both boxes
            df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))
            df_box2 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))

            # Process both boxes in one loop
            eq_data_dict = {}
            for b, box in enumerate([df_box1, df_box2]):
                box_name = "Liquid" if b == 0 else "Vapor"
                for prop_index in prop_cols:
                    eq_col = box[:, prop_index - 1]
                    #Save eq_col as a csv for later analysis
                    key = f"{box_name}_{prop_index}"
                    eq_col_file = job.fn(f"{box_name}_eq_col_{prop_index}.csv")
                    # np.savetxt(eq_col_file, eq_col, delimiter=",")
                    #Save the eq_col and file to a dictionary for later use
                    eq_data_dict[key] = {"data": eq_col, "file": eq_col_file}

            prod_tol_eq = int(eq_data_dict[key]["data"].size/4)
            print(f"Prod Tol: {prod_tol_eq}")
            #While we are using at most 12 attempts to equilibrate
            while count <= 13:
                # Check if equilibration is reached via the pymser algorithms
                is_equil = check_equil_converge(job, eq_data_dict, prod_tol_eq)
                #If equilibration is reached, break the loop and start production
                if is_equil:
                    break
                else:
                    #Increase the total number of eq steps by 25% of the original value and restart the simulation
                    total_eq_steps += int(prod_tol)
                    #If we've exceeded the maximum number of equilibrium steps, raise an exception
                    #This forces a retry with critical conditions or will note complete GEMC failure
                    if count == 13:
                        # job.doc.equil_fail = True
                        use_crit = True
                        raise Exception(f"GEMC equilibration failed to converge after {job.sp.nsteps_eq*4} steps")
                    #Otherwise continue equilibration
                    else:
                        #Add restart data to eq_col
                        # After each restart, load the updated properties data for both boxes
                        # sim_box1 =  "gemc.eq" + f".rst.{count:03d}" + ".out.box1.prp"
                        sim_box2 =  "gemc.eq" + f".rst.{count:03d}" + ".out.box2.prp"
                        print(sim_box2)
                        # df_box1r = np.genfromtxt(job.fn(sim_box1))
                        # df_box2r = np.genfromtxt(job.fn(sim_box2))
                        #Use fake data to extend simulation
                        df_box1r = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))[int(3*len(df_box1)/4):]
                        df_box2r = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))[int(3*len(df_box2)/4):]

                        
                        # df_box1 = np.genfromtxt(job.fn("gemc.eq.out.box1.prp"))
                        # df_box2 = np.genfromtxt(job.fn("gemc.eq.out.box2.prp"))

                        # Process and add the restart data to eq_col for each property in each box
                        for b, box in enumerate([df_box1r, df_box2r]):
                            box_name = "Liquid" if b == 0 else "Vapor"
                            for i, prop_index in enumerate(prop_cols):
                                #Get the key from the property and box name
                                key = f"{box_name}_{prop_index}"
                                # Extract the column data for this restart and append to accumulated data
                                eq_col_restart = box[:, prop_index - 1]
                                print("Reastart len", len(eq_col_restart))
                                eq_col_data = eq_data_dict[key]["data"]
                                all_eq_data = np.concatenate((eq_col_data, eq_col_restart))
                                #Save the new data to the eq_col file
                                print(eq_data_dict[key]["file"])
                                # np.savetxt(eq_data_dict[key]["file"], all_eq_data, delimiter=",")
                                #Overwite the current data in the eq_data_dict with restart data
                                print("orig len", len(eq_data_dict[key]["data"]))
                                # print(eq_data_dict[key]["data"][0:5])
                                eq_data_dict[key]["data"] = all_eq_data
                                print("totla len", len(eq_data_dict[key]["data"]))
                #Increase the counter
                count += 1

            #Set the step counter to whatever the final number of equilibration steps was
            total_steps = total_eq_steps
            print(len(eq_data_dict["Liquid_5"]["data"]))
            print(f"Total steps: {total_steps}")
            # job.doc.equil_fail = False

            # Run production
            print("Running production")
    except:
        # if GEMC failed with critical conditions as intial conditions, terminate with error
        if use_crit == True:
            # If so, terminate with error and log failure in job document
            # job.doc.gemc_failed = True
            print("GEMC failed with critical and experimental starting conditions and the molecule is " + job.sp.mol_name + " at temperature " + str(job.sp.T))
            # raise Exception(
            #     "GEMC failed with critical and experimental starting conditions and the molecule is "
            #     + job.sp.mol_name
            #     + " at temperature "
            #     + str(job.sp.T)
            # )
        else:
            print("Reatart w/ Critical Conditions")
            # Otherwise, try with critical conditions
            # job.doc.use_crit = True
            # # Ensure that you will do an nvt simulation before the next gemc simulation
            # job.sp.nsteps_nvt = 2500000
            # # If GEMC fails, remove files in post conditions of previous operations
            # del job.doc["vapboxl"]  # calc_boxes
            # del job.doc["liqboxl"]  # calc_boxes
            # with job:
            #     if job.isfile("nvt.eq.out.prp"):
            #         os.remove("nvt.eq.out.prp")  # NVT_liqbox
            #         os.remove("nvt.final.xyz")  # extract_final_NVT_config
            #     if job.isfile("npt.eq.out.prp"):
            #         os.remove("npt.eq.out.prp")  # NPT_liqbox
            #         os.remove("npt.final.xyz")  # extract_final_NPT_config
            #     if "liqbox_final_dim" in job.doc:
            #         del job.doc["liqbox_final_dim"]  # extract_final_NPT_config
            #         os.remove("liqbox.xyz")  # extract_final_NPT_config

In [ ]:
import glob
mol_name = "R23"
project = signac.get_project("opt_ff_ms")
jobs = project.find_jobs({"mol_name": mol_name, "T": 240, "atom_type": 8})
# jobs = project.find_jobs({"mol_name": mol_name})
# job = list(project.find_jobs({"mol_name": mol_name, "T": 290, "atom_type": 6}))[0]
# job = list(project.find_jobs({"mol_name": mol_name, "T": 230, "atom_type": 1}))[0]
# print(job)
# for job in jobs:
#     print("ID", job.id, "AT", job.sp.atom_type, "T", job.sp.T)
#     run_gemc(job)
run_gemc(job)